<a href="https://colab.research.google.com/github/emmanuelmokel/PPL-practice/blob/main/NestedRHat_TFP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!rm -Rf probability
!rm -Rf fun_mc
!rm -Rf inference_gym
!git clone https://github.com/tensorflow/probability.git
!mv probability/spinoffs/fun_mc/fun_mc .
!mv probability/spinoffs/inference_gym/inference_gym .
!pip install tf-nightly tfp-nightly jax jaxlib

!pip install immutabledict

Cloning into 'probability'...
remote: Enumerating objects: 100058, done.
remote: Counting objects: 100% (4841/4841), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 100058 (delta 4729), reused 4697 (delta 4670), pack-reused 95217
Receiving objects: 100% (100058/100058), 136.96 MiB | 19.78 MiB/s, done.
Resolving deltas: 100% (81890/81890), done.
Updating files: 100% (1440/1440), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import numpy as np
from matplotlib.pyplot import *

import jax
from jax import random
from jax import numpy as jnp


#!pip install tfp-nightly inference_gym
from inference_gym import using_jax as gym

#from tensorflow_probability.spinoffs import using_jax as fun_mcmc
from fun_mc import using_jax as fun_mcmc

from tensorflow_probability.python.internal import prefer_static as ps
from tensorflow_probability.python.internal import unnest

import tensorflow_probability as _tfp
tfp = _tfp.substrates.jax
tfd = tfp.distributions
tfb = tfp.bijectors

tfp_np = _tfp.substrates.numpy
tfd_np = tfp_np.distributions

In [17]:
# Working with the 'Banana' Posterior Target Density

target = gym.targets.VectorModel(gym.targets.Banana(),
                                 flatten_sample_transformations = True)
num_dimensions = target.event_shape[0]
# Is the heuristic for the initial value of \epsilon given in the paper used in practice?
init_step_size = 1.

def target_log_prob_fn(x):
   y = target.default_event_space_bijector(x)
   fldj = target.default_event_space_bijector.forward_log_det_jacobian(x)
   return target.unnormalized_log_prob(y) + fldj

offset = 2
def bn_initialize(shape, key = random.PRNGKey(3727709)):
  return 3 * random.normal(key, shape + (num_dimensions,)) + offset

/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/internal/backend/jax/ops.py:285: UserWarning: Explicitly requested dtype float64 requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return arr.astype(dtype)


In [18]:
num_chains = 512
K = 4
num_warmup, num_sampling = 100, 1000
total_samples = num_warmup + num_sampling

kernel = tfp.mcmc.HamiltonianMonteCarlo(target_log_prob_fn, init_step_size, 1)
kernel = tfp.experimental.mcmc.GradientBasedTrajectoryLengthAdaptation(kernel, num_warmup)
kernel = tfp.mcmc.DualAveragingStepSizeAdaptation(kernel, num_warmup,
                                                  target_accept_prob = 0.75,
                                                  reduce_fn = tfp.math.reduce_log_harmonic_mean_exp)

# Initializing each chain (in a super chain) at the same location
initial_state = bn_initialize((K,))
initial_state = np.repeat(initial_state, num_chains // K, axis = 0)

In [19]:
result = tfp.mcmc.sample_chain(total_samples, initial_state,
                               kernel = kernel, seed = random.PRNGKey(1954))

/usr/local/lib/python3.10/dist-packages/tensorflow_probability/substrates/jax/mcmc/sample.py:339: UserWarning: Tracing all kernel results by default is deprecated. Set the `trace_fn` argument to None (the future default value) or an explicit callback that traces the values you are interested in.
  warnings.warn('Tracing all kernel results by default is deprecated. Set '
